In [1]:
import pandas as pd
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [2]:
from pyathena import connect

In [3]:
ingest_create_athena_table_csv_passed = False

In [4]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)



In [5]:
%store -r ingest_create_athena_db_passed


## Creating 6 tables in Athena

In [6]:
s3_private_path_csv1 = "s3://{}/projectads508/aisles/".format(bucket)
print(s3_private_path_csv1)

s3://sagemaker-us-east-1-254716743917/projectads508/aisles/


In [7]:
s3_private_path_csv2 = "s3://{}/projectads508/departments/".format(bucket)
print(s3_private_path_csv2)

s3://sagemaker-us-east-1-254716743917/projectads508/departments/


In [8]:
s3_private_path_csv3 = "s3://{}/projectads508/prior/".format(bucket)
print(s3_private_path_csv3)

s3://sagemaker-us-east-1-254716743917/projectads508/prior/


In [9]:
s3_private_path_csv4 = "s3://{}/projectads508/train/".format(bucket)
print(s3_private_path_csv4)

s3://sagemaker-us-east-1-254716743917/projectads508/train/


In [10]:
s3_private_path_csv5 = "s3://{}/projectads508/orders/".format(bucket)
print(s3_private_path_csv5)

s3://sagemaker-us-east-1-254716743917/projectads508/orders/


In [11]:
s3_private_path_csv6 = "s3://{}/projectads508/products/".format(bucket)
print(s3_private_path_csv6)

s3://sagemaker-us-east-1-254716743917/projectads508/products/


In [12]:
# Set Athena parameters
table1 = "aisles"
table2= "departments"
table3= "order_products_prior"
table4= "order_products_train"
table5= "orders"
table6= "products"
database_name='oneclickaws'

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [14]:
# SQL statement to execute aisles table
statement1 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         aisle_id int,
         aisle string
       
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table1, s3_private_path_csv1
)


pd.read_sql(statement1, conn)

""


In [15]:
# SQL statement to execute department table
statement2 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         department_id int,
         department string
       
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table2, s3_private_path_csv2
)


pd.read_sql(statement2, conn)

""


In [16]:
# SQL statement to execute order_products__prior table
statement3 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         order_id int,
         product_id int,
         add_to_cart_order int,
         reordered string
       
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table3, s3_private_path_csv3
)


pd.read_sql(statement3, conn)

""


In [17]:
# SQL statement to execute order_products__train table
statement4 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         order_id int,
         product_id int,
         add_to_cart_order int,
         reordered string
       
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table4, s3_private_path_csv4
)


pd.read_sql(statement4, conn)

""


In [18]:
# SQL statement to execute orders table
statement5 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         order_id int,
         user_id int,
         eval_set string,
         order_number int,
         order_dow string,
         order_hour_of_day string,
         days_since_prior_order float
       
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table5, s3_private_path_csv5
)


pd.read_sql(statement5, conn)

""


In [19]:
# SQL statement to execute products table
statement6= """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
          product_id int,
         product_name string,
         aisle_id int,
         department_id int
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table6, s3_private_path_csv6
)


pd.read_sql(statement6, conn)

""


## Test created database and tables

In [20]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(6)

,tab_name
0,aisles
1,departments
2,order_products_prior
3,order_products_train
4,orders
5,products


In [21]:
test_statement = """SELECT * FROM {}.{} LIMIT 5 """.format(
    database_name, table1
)

print(test_statement)
df_show = pd.read_sql(test_statement, conn)
df_show.head(5)

SELECT * FROM oneclickaws.aisles LIMIT 5 


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [22]:
test_statement = """SELECT * FROM {}.{} LIMIT 5 """.format(
    database_name, table2
)

print(test_statement)
df_show = pd.read_sql(test_statement, conn)
df_show.head(5)

SELECT * FROM oneclickaws.departments LIMIT 5 


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [23]:
test_statement = """SELECT * FROM {}.{}  order by order_id LIMIT 12  """.format(
    database_name, table3
)

print(test_statement)
df_show = pd.read_sql(test_statement, conn)
df_show.head(12)

SELECT * FROM oneclickaws.order_products_prior  order by order_id LIMIT 12  


,order_id,product_id,add_to_cart_order,reordered
0,2,1819,8,1
1,2,9327,3,0
2,2,45918,4,1
3,2,40141,7,1
4,2,43668,9,0
5,2,30035,5,0
6,2,17794,6,1
7,2,28985,2,1
8,2,33120,1,1
9,3,33754,1,1


In [24]:
test_statement = """SELECT * FROM {}.{} LIMIT 5 """.format(
    database_name, table4
)

print(test_statement)
df_show = pd.read_sql(test_statement, conn)
df_show.head(5)

SELECT * FROM oneclickaws.order_products_train LIMIT 5 


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [25]:
test_statement = """SELECT * FROM {}.{} order by user_id LIMIT 12 """.format(
    database_name, table5
)

print(test_statement)
df_show = pd.read_sql(test_statement, conn)
df_show.head(12)

SELECT * FROM oneclickaws.orders order by user_id LIMIT 12 


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1187899,1,train,11,4,08,14.0
1,3367565,1,prior,6,2,07,19.0
2,3108588,1,prior,8,1,14,14.0
3,2550362,1,prior,10,4,08,30.0
4,473747,1,prior,3,3,12,21.0
5,431534,1,prior,5,4,15,28.0
6,2539329,1,prior,1,2,08,NaN
7,550135,1,prior,7,1,09,20.0
8,2398795,1,prior,2,3,07,15.0
9,2295261,1,prior,9,1,16,0.0


In [26]:
test_statement = """SELECT * FROM {}.{} LIMIT 5 """.format(
    database_name, table6
)

print(test_statement)
df_show = pd.read_sql(test_statement, conn)
df_show.head(5)

SELECT * FROM oneclickaws.products LIMIT 5 


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [27]:
%store

Stored variables and their in-db values:
database_name                              -> 'oneclickaws'
ingest_create_athena_db_passed             -> True
s3_public_path_csv                         -> 's3://projectads508'
